<a href="https://colab.research.google.com/github/j7tfj7f8k1f/LLMCOSPLAY-RAG-TTS-/blob/main/LLMCOSPLAY(RAG%2BTTS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 歡迎來到實現LLMCOSPLAY的筆記本
#首先我必須要非常感謝Artrajz的vits-simple-api程式碼
#有他的API介面省了非常多的時間



# 前置作業GOOGLE_API_KEY = gemini api的金鑰
#HF_TOKEN = huggingface的金鑰
# 請到這裡申請 https://aistudio.google.com/app/apikey
# 接下來請到他的API介面https://colab.research.google.com/drive/1uBkMy0UjLE3C1zvxZ7NPPc3K74v4B6zw

# 運行完會拿到類似像API_URL = "https://4cf4-54-157-39-105.ngrok-free.app/voice/vits"  這樣的分享連接


# 如果還沒有訓練聲音模型的 推薦用Plachtaa/VITS-fast-fine-tuning
# https://colab.research.google.com/drive/1pn1xnFfdLK63gVXDwV4zCXfVeo8c-I-0?usp=sharing

## <<步驟1>> 工具跟聊天機器人顯示的圖片

In [1]:
!pip install -q -U google-generativeai
!pip install faiss-cpu
!pip install sentence-transformers
!pip install gradio
!pip install soundfile
!pip install requests
!pip install pykakasi
!pip install pydub
!sudo apt-get install ffmpeg
!pip install chess

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.1 MB/s eta 0:00:00
  Created wheel for jaconv: filename=jaconv-0.4.0-py3-none-any.whl size=18229 sha256=8f61abbb9f6f77d1f6df8798f3276d39069c8375a1fa1bcc28da87babe5ba6ae
  Stored in 

In [2]:
!gdown 11tjI0HFW_DHuRm4bz7h1QVvGnruW_a5G

Downloading...
From: https://drive.google.com/uc?id=11tjI0HFW_DHuRm4bz7h1QVvGnruW_a5G
To: /content/soyo.png
100% 1.44M/1.44M [00:00<00:00, 24.8MB/s]


## <<步驟2>> 創造RAG向量資料庫 請依照以下格式填入或是直接上傳符合格式的 以下只是示範 實測大概要70組左右才OK 如果要放劇情請差不多每6~7段一組 像是"title": "劇情1", "title": "劇情2"....... 不然太長

In [3]:
# @title
%%writefile medical_knowledge.json

[
  {
    "title": "全名",
    "content": "長崎そよ"
  },
  {
    "title": "個性",
    "content": "傲嬌大小姐"
  },
  {
    "title": "名言",
    "content": "為什麼要演奏春日影!!!!!"
  }
]


Writing medical_knowledge.json


## <<步驟3>> 我們採用intfloat的中文Embedding模型跟faiss的向量檢索庫 如果對token數不滿意可以參考這個評比網站 https://ihower.tw/blog/archives/12167

In [4]:
from sentence_transformers import SentenceTransformer
import faiss
import json

# 加載語義嵌入模型
model = SentenceTransformer('intfloat/multilingual-e5-large')

# 加載資料集
with open('medical_knowledge.json', 'r') as f:
    data = json.load(f)

# 計算向量嵌入
texts = [item['content'] for item in data]
embeddings = model.encode(texts)

# 建立向量檢索庫
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# 存儲檢索庫
faiss.write_index(index, 'medical_index.faiss')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

## <<步驟4>> 開啟gradio介面 請將vits api拿到的分享連接貼入以下2組程式碼內

# 由於我們的vits模型是訓練日文發音的 但我們打的問題是中文 因此必須限制他回應是日文 並且不能漢字 因為漢字在TTS裡面很容易被當中文

In [6]:
import os
import json
import google.generativeai as genai
from google.colab import userdata
from sentence_transformers import SentenceTransformer
import faiss
import gradio as gr
import requests
import io
from pydub import AudioSegment
import pykakasi

# 設定 Google API 金鑰
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))
# VITS API 網址
API_URL = "https://4cf4-54-157-39-105.ngrok-free.app/voice/vits"

# 加載檢索庫和模型
model = SentenceTransformer('intfloat/multilingual-e5-large')
index = faiss.read_index('medical_index.faiss')

# 加載資料集
with open('medical_knowledge.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 定義檢索函數
def retrieve(query, k=3):
    query_vec = model.encode([query])
    distances, indices = index.search(query_vec, k)
    return indices[0]  # 返回最相關的段落索引

# 對話歷史檔案路徑
conversation_history_file = '/content/conversation_history.txt'

# 載入對話歷史
def load_conversation_history():
    try:
        with open(conversation_history_file, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        return ""

# 儲存對話歷史
def save_conversation_history(history):
    with open(conversation_history_file, 'w', encoding='utf-8') as f:
        f.write(history)
# VITS 文字轉語音函數
def text_to_speech(text, speaker_id):
    params = {"id": speaker_id, "text": text}
    response = requests.get(API_URL, params=params)

    if response.status_code == 200:
        audio_data = io.BytesIO(response.content)
        audio = AudioSegment.from_file(audio_data, format="wav")
        output_path = "output.wav"
        audio.export(output_path, format="wav")
        return output_path
    else:
        return f"Error: {response.status_code}, {response.text}"


# 從 `/voice/speakers` 獲取角色列表
speakers_response = requests.get("https://4cf4-54-157-39-105.ngrok-free.app/voice/speakers")
if speakers_response.status_code == 200:
    speakers = speakers_response.json()
    speaker_choices = {name: name for name in speakers.keys()}
else:
    speaker_choices = {"Default": "Default"}

default_speaker = list(speaker_choices.keys())[0] if speaker_choices else "Default"
# 漢字轉平假名函數
def kanji_to_hiragana(text):
    kakasi = pykakasi.kakasi()
    kakasi.setMode("H", "H")  # Hiragana to Hiragana (不變)
    kakasi.setMode("K", "H")  # Katakana to Hiragana
    kakasi.setMode("J", "H")  # Japanese to Hiragana
    conv = kakasi.getConverter()
    result = conv.do(text)
    return result
# 問答函數
def answer_question(question):
    # 載入對話歷史
    full_conversation_history = load_conversation_history()

    # 檢索相關段落
    indices = retrieve(question)
    context = " ".join([data[i]['content'] for i in indices])

    # 組合包含完整對話歷史的提示
    prompt = f"你是一名叫做爽世的高中生, 在樂團中是個貝斯手 個性端莊淑女 並且因為是為日本人 只能回應日文。請基於以下上下文和之前的對話歷史回答問題。\n\n對話歷史：\n{full_conversation_history}\n\n上下文：\n{context}\n\n問題：\n{question}"

    # 使用 Gemini 生成回答
    model = genai.GenerativeModel("gemini-1.5-flash")  # 或 "gemini-1.5-flash-turbo"
    response = model.generate_content([prompt])
    answer = response.text
    # 將漢字轉換為平假名
    answer = kanji_to_hiragana(answer)
    # 將回答傳送到 VITS 產生語音
    audio_path = text_to_speech(answer, default_speaker) # 使用預設的 speaker

    # 更新完整對話歷史
    full_conversation_history += f"使用者：{question}\n爽世：{answer}\n"

    # 儲存對話歷史
    save_conversation_history(full_conversation_history)

    return answer, audio_path

# 追加資料到 medical_knowledge.json
def append_data(title, content):
    new_data = {"title": title, "content": content}
    with open('medical_knowledge.json', 'r+', encoding='utf-8') as f:
        try:
            existing_data = json.load(f)
        except json.JSONDecodeError: # 處理空檔案
            existing_data = []
        existing_data.append(new_data)
        f.seek(0)  # 回到檔案開頭
        json.dump(existing_data, f, indent=2, ensure_ascii=False)
        f.truncate() # 清除後續內容
        global data #更新全域變數data
        data = existing_data
        # 重新建立索引
        embeddings = model.encode([item['content'] for item in data])
        index = faiss.IndexFlatL2(embeddings.shape[1])
        index.add(embeddings)
        faiss.write_index(index, "medical_index.faiss")


    return "資料已成功新增！"

def clear_history():  # Add this function definition
    save_conversation_history("")  # This will clear the history file
    return []
# 建立 Gradio 介面
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="輸入你的問題:")
    clear = gr.Button("清除歷史紀錄")
    audio_output = gr.Audio(label="語音回覆")

    with gr.Row():
        title_input = gr.Textbox(label="標題")
        content_input = gr.Textbox(label="內容")
        append_button = gr.Button("新增資料 (很慢，大約一次 40 秒)")

    #  載入爽世圖片
    soyo_image = gr.Image(value="/content/soyo.png", visible=False)

    def user(user_message, history):
        response, audio_path = answer_question(user_message)
        bot_message = f"<img src='{soyo_image.value['url']}' width='3' height='5' style='vertical-align:middle;'> {response}"
        history.append((user_message, bot_message))
        return "", history, audio_path

    msg.submit(user, [msg, chatbot], [msg, chatbot, audio_output]) # 更新輸出目標，包含音訊
    clear.click(clear_history, outputs=chatbot)
    append_button.click(append_data, inputs=[title_input, content_input], outputs=gr.Textbox(label="狀態"))

demo.launch(debug=True)




/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b2fbebb758ff1da44e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b2fbebb758ff1da44e.gradio.live
